In [21]:
import tensorflow as tf
import os
import tfx
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip



The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [22]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")

In [23]:
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [24]:
example_gen = ImportExampleGen(input_base=tf_record_location)#ImportExampleGen because we are impoting Tfrecord file format

In [25]:
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx_for_tfrecordData')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 298
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [26]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 299
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [27]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 300
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Transform

In [28]:
# Set the transform module filename
_housing_transform_module_file = 'housing_transform.py'

In [29]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'], # tfExample
    schema=schema_gen.outputs['schema'],# Schema
    module_file=os.path.abspath(_housing_transform_module_file)) # preprocessing_fn()



In [30]:
# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 301
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Tuner

In [31]:
_tuner_module_file = 'tuner.py'

In [32]:
# Setup the Tuner component  /// https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Tuner
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=512),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [33]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)

Trial 30 Complete [00h 00m 31s]
val_loss: 0.9934375286102295

Best val_loss So Far: 0.9831249713897705
Total elapsed time: 00h 05m 26s
Results summary
Results in ../pipeline_root_learning/tfx_for_tfrecordData\.temp\302\housing_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0005 summary
Hyperparameters:
units_1: 50
units_2: 250
units_3: 375
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9831249713897705

Trial 0012 summary
Hyperparameters:
units_1: 50
units_2: 250
units_3: 375
learning_rate: 0.01
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0005
Score: 0.9846875071525574

Trial 0016 summary
Hyperparameters:
units_1: 50
units_2: 250
units_3: 375
learning_rate: 0.01
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.9865624904632568

Trial 0023 summary
Hyperparameters:
units_1: 50
units_2: 225
units_3: 350
learn

TypeError: '<' not supported between instances of 'int' and 'NoneType'

## Trainer

In [34]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [35]:
# Import necessary libraries
# Import necessary libraries
from tensorflow import keras
from kerastuner.engine import base_tuner
import os
from pathlib import Path
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Constants from housing_constants_module_file
import housing_constant
_transformed_name = housing_constant.transformed_name

# defining a structure(NamedTuple) that can hold both the tuner tool (base_tuner) and the training instructions(fit_kwargs) together
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])
# defining the label
LABEL_KEY = "ocean_proximity_xf"




# 1. Load compressed dataset
def _gzip_reader_fn(filenames):  # helper function to load data 
  '''Load compressed dataset

  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''
#Loading the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')




# 2. Data loading for model training and validation is performed in batches, and the loading is handled by the input_fn()
def _input_fn(file_pattern,     # helper function to load data and preprocessing 
              tf_transform_output,
              num_epochs = None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output -The transform output graph from TensorFlow Transform (TF Transform) that contains information about how features have been transformed or preprocessed.
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''

  # Get feature specification based on transform output = This specification defines the structure and types of features after they have been transformed by TF Transform.
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset
# The input_fn returns a generator (a batched_features_dataset) that will supply
# data to the model one batch at a time.



# 3. Applying the preprocessing graph to model inputs
# This function bridges the gap between your trained model and its deployment for serving predictions. 
# It essentially defines how the model handles incoming requests containing new data points.

#Signatures specify how the model can be used for inference, including input and output formats.
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    model.tft_layer = tf_transform_output.transform_features_layer() #Load the preprocessing graph

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(housing_constant.LABEL_KEY)
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec  #Parse the raw tf.Example records from the request.
        )

        transformed_features = model.tft_layer(parsed_features) #Apply the preprocessing transformation to raw data

        outputs = model(transformed_features) #Perform prediction with preprocessed data.
        return {"outputs": outputs}

    return serve_tf_examples_fn




# 4. DEFINING MODEL ARCHITECTURE
def build_model(hp):
  """Builds a Keras model for housing price prediction with hyperparameter tuning.

  Args:
      hp: Hyperparameters object from KerasTuner.

  Returns:
      A compiled Keras model.
  """

  input_features = []
  for x in housing_constant.NUMERIC_FEATURE_KEYS:
    input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))

  for x in housing_constant.BUCKET_FEATURE_KEYS:
    input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))  # Bucket features have shape (1,) after transformation

  # Concatenate all input layers
  concat = tf.keras.layers.concatenate(input_features)

  # Hidden layers
  first_layer = tf.keras.layers.Dense(units=hp.get('units_1'), activation='relu')(concat)
  dropout_layer = tf.keras.layers.Dropout(0.2)(first_layer)
  second_layer = tf.keras.layers.Dense(units=hp.get('units_2'), activation='relu')(dropout_layer)
  third_layer = tf.keras.layers.Dense(units=hp.get('units_3'), activation='relu')(second_layer)

  # Output layer
  output_layer = tf.keras.layers.Dense(5, activation='softmax')(third_layer)

  # Create the model
  model = tf.keras.Model(inputs=input_features, outputs=output_layer)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.get('learning_rate') 
                                                ) ,loss='sparse_categorical_crossentropy',metrics=['categorical_accuracy'])
  # Print model summary for debugging purposes (optional)
  model.summary()

  return model




# 5. Training Orchestration
# run_fn is the main function that coordinates training. It calls functions in a specific order to:
# Load the transform graph (tf_transform_output)
# Prepare training and validation datasets using _input_fn with the transform graph
# Load hyperparameters from fn_args
# Build the model using build_model with loaded hyperparameters
# Train the model using model.fit with prepared datasets and callbacks
# Save the model with signatures using model.save


#The Trainer component will look for a run_fn() function in our module file and use
#the function as an entry point to execute the training process. 
#The module file needs The TFX Trainer Component to be accessible to the Trainer component

def run_fn(fn_args: FnArgs) -> None:
  """Defines and trains the model.
  Args:
    fn_args: Holds args as name/value pairs. fn_args: This is an object of type FnArgs that holds arguments as name/value pairs. 
    It contains various attributes such as paths to transform graphs, example datasets, hyperparameters, and model directories
    Refer here for the complete attributes: 
    https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
  """

#the run_fn function receives a set of arguments, including the transform
#graph, example datasets, and training parameters through the fn_args object
    
  # 5.1 Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  # Create batches of data good for 10 epochs
  train_set = _input_fn(fn_args.train_files, tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files, tf_transform_output, 10)

  # 5.2 Load best hyperparameters
  hp = fn_args.hyperparameters.get('values')
    

  # 5.3 Build the model
  model = build_model(hp)

  #Callbacks
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir, update_freq='batch')
    
  # 5.4 Train the model
  model.fit(x=train_set,validation_data=val_set , callbacks=[tensorboard_callback])
#Signatures specify how the model can be used for inference, including input and output formats.
  signatures = {'serving_default': 
                _get_serve_tf_examples_fn(model,tf_transform_output).get_concrete_function(tf.TensorSpec(shape=[None],
                                                                                                         dtype=tf.string,name='examples'))}
  # 5.5 Save the model
  model.save(fn_args.serving_model_dir, save_format='tf')  #,signatures=signatures
  model_name = "my_housing_model"
  model_version = "0003"
  model_path = Path(model_name) / model_version   #another method to save the model
  model.save(model_path, save_format="tf")


Overwriting trainer.py


• The transformed examples generated by the Transform component

• The transform graph generated by the Transform component

• The schema generated by the data validation component

• The number of training and evaluation steps

In [36]:
# Setup the Trainer component
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']))

In [37]:
# Run the component
context.run(trainer, enable_cache=False)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 longitude_xf (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 latitude_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 total_rooms_xf (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 total_bedrooms_xf (InputLa  [(None, 1)]                  0         []                      

ExecutionResult(
    component_id: Trainer
    execution_id: 303
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [18]:
# Get artifact uri of trainer model output
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List subdirectories artifact uri
print(f'contents of model artifact directory:{os.listdir(model_artifact_dir)}')

# Define the model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List contents of model directory
print(f'contents of model directory: {os.listdir(model_dir)}')

contents of model artifact directory:['Format-Serving', 'logs']
contents of model directory: ['assets', 'fingerprint.pb', 'keras_metadata.pb', 'saved_model.pb', 'variables']


In [19]:
#  model_artifact_dir = trainer.outputs['model'].get()[0].uri
# # log_dir = os.path.join(model_artifact_dir, 'logs')
# log_dir = '../pipeline_root_learning/tfx_for_tfrecordData/Trainer/model/220/logs'
# %reload_ext tensorboard
# %tensorboard --logdir {log_dir} 

## Using the SavedModel outside a pipeline

###### In a complete pipeline, if you're using TensorFlow Serving for inference, you typically don't need to load the model manually using tf.saved_model.load as shown above. TensorFlow Serving handles the loading and serving of the model automatically based on the configuration you provide.

In [20]:
# If we would like to inspect the exported SavedModel outside a TFX pipeline, we can
# load the model as a concrete function,5 which represents the graph of a single
# signature:
model = tf.saved_model.load(export_dir=model_dir)
predict_fn = model.signatures["serving_default"]
predict_fn

<ConcreteFunction signature_wrapper(*, examples) at 0x108320C12B0>

In [34]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

# Prepare your input data (assuming you have a single data point for prediction)
input_data = {
    "longitude": 11111110,
    "latitude": 1111110,
    "housing_median_age": 111110,
    "total_rooms": 111110,
    "total_bedrooms":145011111,
    "population": 45455440,
    "households": 4554540,
    "median_income": 4545450.3252,
    "median_house_value":454545540
}

# Convert input data to tf.train.Example format
example = tf.train.Example(features=tf.train.Features(feature={
    "longitude": _float_feature(input_data["longitude"]),
    "latitude": _float_feature(input_data["latitude"]),
    "housing_median_age": _float_feature(input_data["housing_median_age"]),
    "total_rooms": _float_feature(input_data["total_rooms"]),
    "total_bedrooms": _float_feature(input_data["total_bedrooms"]),
    "population": _float_feature(input_data["population"]),
    "households": _float_feature(input_data["households"]),
    "median_income": _float_feature(input_data["median_income"]),
    "median_house_value": _float_feature(input_data["median_house_value"])
}))

# Serialize the example
serialized_example = example.SerializeToString()

In [35]:
serialized_example

b'\n\xed\x01\n\x1a\n\x0etotal_bedrooms\x12\x08\x12\x06\n\x04\x1aK\nM\n\x1e\n\x12housing_median_age\x12\x08\x12\x06\n\x04\x00\x03\xd9G\n\x17\n\x0btotal_rooms\x12\x08\x12\x06\n\x04\x00\x03\xd9G\n\x1e\n\x12median_house_value\x12\x08\x12\x06\n\x04\x84\xbe\xd8M\n\x14\n\x08latitude\x12\x08\x12\x06\n\x040\xa2\x87I\n\x19\n\rmedian_income\x12\x08\x12\x06\n\x04U\xb7\x8aJ\n\x16\n\nhouseholds\x12\x08\x12\x06\n\x04X\xfe\x8aJ\n\x16\n\npopulation\x12\x08\x12\x06\n\x04\x14f-L\n\x15\n\tlongitude\x12\x08\x12\x06\n\x04\xc6\x8a)K'

In [36]:
print(predict_fn(tf.constant([serialized_example])))

{'outputs': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>}
